# Libaries

In [ ]:
from keras import callbacks
from keras.models import Sequential
from keras.layers import Activation,Flatten,Dense,Dropout,Embedding,Bidirectional,LSTM
from keras.optimizers import Adam,SGD
import matplotlib.pyplot as plt

import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Bidirectional,SpatialDropout1D
from keras.layers.embeddings import Embedding

from sklearn.model_selection import KFold
import gc
import numpy as np
import keras.backend as K

import numpy as np
import pandas as pd
import seaborn as sb
import math

import os
import re
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout
#from kaggle_datasets import KaggleDatasets
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Model declaration

In [ ]:
"""
model declaration and callbacks declaration
callbacks are used to exercise control over the model. For instance, callbacks can be used to perform early stopping, model check-pointing,
lr reduction over time,
"""
def create_model(vocabulary_size,embedding_size,embedding_matrix):
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix], trainable=False))
    model_glove.add(Bidirectional(LSTM(100)))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_glove.summary()
    return model_glove


def callback(model_name,tf_log_dir_name='./tf-log/',patience_lr=10,):
    cb = []
    """
    Tensorboard log callback
    """
    tb = callbacks.TensorBoard(log_dir=tf_log_dir_name, histogram_freq=0)
    cb.append(tb)

    """
    Model-Checkpoint
    """
    m = callbacks.ModelCheckpoint(filepath=model_name,monitor='val_loss',mode='auto',save_best_only=True)
    cb.append(m)

    """
    Reduce Learning Rate
    """
    reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    cb.append(reduce_lr_loss)

    """
    Early Stopping callback
    """
    # Uncomment for usage
    early_stop = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
    cb.append(early_stop)



    return cb

######### Show Train Val History Graph ###############
def plot_loss_accu(history,lossLoc='Train_Val_Loss',accLoc='Train_Val_acc'):

    plt.clf()

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.legend(['train', 'val'], loc='upper right')
    #plt.show()
    plt.savefig(lossLoc)

    plt.clf()

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and validation accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    #plt.show()
    plt.savefig(accLoc)


    return model_glove

# Preprocessing Functions

In [ ]:
!git clone https://github.com/banglakit/bengali-stemmer.git

Cloning into 'bengali-stemmer'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 98 (delta 6), reused 15 (delta 3), pack-reused 69
Unpacking objects: 100% (98/98), done.


In [ ]:
cd "/content/bengali-stemmer"

/content/bengali-stemmer


In [ ]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('কেজিতে')

In [ ]:
!pip install bnlp_toolkit

     |████████████████████████████████| 23.9MB 121kB/s 
     |████████████████████████████████| 1.2MB 44.5MB/s 
     |████████████████████████████████| 747kB 34.9MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from bnlp.corpus import stopwords, punctuations
stopwordsBNLP = stopwords
print(len(stopwordsBNLP))
print(punctuations)

In [ ]:
from bnlp.corpus.util import remove_stopwords
from bnlp import NLTKTokenizer

def removeForeign(word):
  a = "".join(i for i in word if 2432 <= ord(i) <= 2559)
  return a

def makeRemoveHyperLink(text):
  result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
  return result

def MakeHTMLremove(text):
  '''
  result = re.compile('(<.*?>)')
  result = result.sub(r'', text) 
  '''
  cleancode = re.compile('<code>.*?</code>')
  cleanr = re.compile('<.*?>')
  cleanentity = re.compile('&.*;')
  cleantext = re.sub(cleancode, '', text)
  cleantext = re.sub(cleanr, ' ', cleantext)
  cleantext = re.sub(cleanentity, ' ', cleantext)
  
  return cleantext

def cleaning(updated):
  
  #html remove
  updated= updated.apply(lambda x: MakeHTMLremove(x))
  #hyperlink remove
  updated = updated.apply(lambda x: makeRemoveHyperLink(x))
  # tokenizer
  bnltk = NLTKTokenizer()
  updated = updated.apply(lambda x: bnltk.word_tokenize(x))
  # remove punctuations
  updated = updated.apply(lambda x: [item for item in x if item not in punctuations])
  # remove stop words
  updated = updated.apply(lambda x: [item for item in x if item not in stopwordsBNLP])
  # remove foreign words
  updated = updated.apply(lambda x: [ removeForeign(item) for item in x ])
  # stripping
  updated = updated.apply(lambda x: [item.strip() for item in x ])
  # remove numbers
  updated = updated.apply(lambda x: [re.sub(r'[০১২৩৪৫৬৭৮৯\.]+', '', item) for item in x ])
  # stemming
  updated = updated.apply(lambda x: [stemmer.stem_word(item) for item in x ])
  # stripping
  updated = updated.apply(lambda x: [item.strip() for item in x ])
  
  
  return updated

# Data Loading and consolidation

In [ ]:
df1= pd.read_csv('/content/drive/MyDrive/FakeNews/Authentic-48K.csv', engine='python', encoding='utf-8')
print("df1.shape: ",df1.shape)
df2= pd.read_csv('/content/drive/MyDrive/FakeNews/Fake-1K.csv', engine='python', encoding='utf-8')
print("df2.shape: ",df2.shape)
df3= pd.read_csv('/content/drive/MyDrive/FakeNews/LabeledAuthentic-7K.csv', engine='python', encoding='utf-8')
print("df3.shape: ",df3.shape)
df4= pd.read_csv('/content/drive/MyDrive/FakeNews/LabeledFake-1K.csv', engine='python', encoding='utf-8')
print("df4.shape: ",df4.shape)
merged = pd.concat([df1, df2, df3, df4])
print("merged shape: ",merged.shape)
#shuffle
merged = merged.sample(frac=1).reset_index(drop=True)
print("merged columns: ",merged.columns)

df1.shape:  (48678, 7)
df2.shape:  (1299, 7)
df3.shape:  (7202, 9)
df4.shape:  (1299, 10)
merged shape:  (58478, 10)
merged columns:  Index(['articleID', 'domain', 'date', 'category', 'headline', 'content',
       'label', 'source', 'relation', 'F-type'],
      dtype='object')


In [ ]:
merged.info(memory_usage="deep")
print("\nNon null rows count\n")
merged.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58478 entries, 0 to 58477
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   articleID  58478 non-null  int64  
 1   domain     58478 non-null  object 
 2   date       58478 non-null  object 
 3   category   58478 non-null  object 
 4   headline   58478 non-null  object 
 5   content    58478 non-null  object 
 6   label      58478 non-null  float64
 7   source     8501 non-null   object 
 8   relation   8501 non-null   object 
 9   F-type     1299 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 233.5 MB

Non null rows count



articleID        0
domain           0
date             0
category         0
headline         0
content          0
label            0
source       49977
relation     49977
F-type       57179
dtype: int64

In [ ]:
merged.drop(['source', 'relation', 'F-type', 'articleID', 'category', 'domain', 'date'], axis=1, inplace=True)
merged.head(10)

,headline,content,label
0,কিশোরীরা এই উদযাপন তুলে রেখেছিল ট্রফিটার জন্য,আসর জুড়ে একের পর এক ম্যাচ জিতেছে বাংলার কিশোরী...,1.0
1,"‘এখন আমি শিল্পী হয়ে গেছি’ ০১ অক্টোবর, ২০১৮ ১৪:২২",সম্পন্ন হল ‘মিস ওয়ার্ল্ড বাংলাদেশ-২০১৮’র আয়োজন...,1.0
2,উম্মুক্ত পরিবেশেও নির্ঝঞ্ঝাট কাজ নিশ্চিত করবে ...,মো. রায়হান কবির : অনেক অফিসেই খোলামেলা পরিবেশে...,1.0
3,সংসদ নির্বাচনের বিস্তারিত ইইউকে জানিয়েছি,বর্তমান সরকারই নির্বাচনকালীন সরকারের দায়িত্বে ...,1.0
4,সড়কে চলাচলের সময় সৃষ্টিকর্তাকে স্মরণ করা জরুরি,নিরাপদ সড়ক চাই (নিসচা) এর কেন্দ্রীয় কমিটির চেয়...,1.0
5,পালুতে অবতরণের অনুমতি পাচ্ছে না বিদেশি ত্রাণবা...,ইন্দোনেশিয়ার ভূমিকম্প ও সুনামিতে বিধ্বস্ত পালু...,1.0
6,আট দিনের মধ্যে ফিলিস্তিনিদের শহর ছাড়ার নির্দেশ...,আট দিনের মধ্যে পশ্চিম তীরের বেদুইন গ্রামের ফিল...,1.0
7,সুনামিতে নিহত ৮৪০ জনকে একসঙ্গে বিদায় জানালো ইন...,মরদেহের রঙিন ব্যাগগুলো পাশাপাশি পড়ে আছে মাত্র ...,1.0
8,অর্ধশতাধিক পৌরসভা-ইউপিতে উপনির্বাচনে ভোট কাল,দেশের অর্ধশতাধিক পৌরসভা ও ইউনিয়ন পরিষদের (ইউপি...,1.0
9,আলোচনায় বসতে মোদিকে ইমরানের চিঠি,পাকিস্তানের প্রধানমন্ত্রী ইমরান খান ভারতের সঙ্...,1.0


In [ ]:
merged['headline'] = merged['headline'].astype(str)
merged['content'] = merged['content'].astype(str)
merged.sample(10)

In [ ]:
print("Unique Content :",len(list(set(merged['content']))))
print("Total rows :",len(merged))
# dropping ALL duplicte values 
merged.drop_duplicates(subset ="content", 
                     keep = "first", inplace = False)
merged.head(10)

Unique Content : 52901
Total rows : 58478


,headline,content,label
0,কিশোরীরা এই উদযাপন তুলে রেখেছিল ট্রফিটার জন্য,আসর জুড়ে একের পর এক ম্যাচ জিতেছে বাংলার কিশোরী...,1.0
1,"‘এখন আমি শিল্পী হয়ে গেছি’ ০১ অক্টোবর, ২০১৮ ১৪:২২",সম্পন্ন হল ‘মিস ওয়ার্ল্ড বাংলাদেশ-২০১৮’র আয়োজন...,1.0
2,উম্মুক্ত পরিবেশেও নির্ঝঞ্ঝাট কাজ নিশ্চিত করবে ...,মো. রায়হান কবির : অনেক অফিসেই খোলামেলা পরিবেশে...,1.0
3,সংসদ নির্বাচনের বিস্তারিত ইইউকে জানিয়েছি,বর্তমান সরকারই নির্বাচনকালীন সরকারের দায়িত্বে ...,1.0
4,সড়কে চলাচলের সময় সৃষ্টিকর্তাকে স্মরণ করা জরুরি,নিরাপদ সড়ক চাই (নিসচা) এর কেন্দ্রীয় কমিটির চেয়...,1.0
5,পালুতে অবতরণের অনুমতি পাচ্ছে না বিদেশি ত্রাণবা...,ইন্দোনেশিয়ার ভূমিকম্প ও সুনামিতে বিধ্বস্ত পালু...,1.0
6,আট দিনের মধ্যে ফিলিস্তিনিদের শহর ছাড়ার নির্দেশ...,আট দিনের মধ্যে পশ্চিম তীরের বেদুইন গ্রামের ফিল...,1.0
7,সুনামিতে নিহত ৮৪০ জনকে একসঙ্গে বিদায় জানালো ইন...,মরদেহের রঙিন ব্যাগগুলো পাশাপাশি পড়ে আছে মাত্র ...,1.0
8,অর্ধশতাধিক পৌরসভা-ইউপিতে উপনির্বাচনে ভোট কাল,দেশের অর্ধশতাধিক পৌরসভা ও ইউনিয়ন পরিষদের (ইউপি...,1.0
9,আলোচনায় বসতে মোদিকে ইমরানের চিঠি,পাকিস্তানের প্রধানমন্ত্রী ইমরান খান ভারতের সঙ্...,1.0


# Clean and prepare dataset

In [ ]:
"""
prepend headline to content
"""
print("headline: ",merged['headline'][200]," content ",merged['content'][200],)
merged['content'] = merged['headline'] +" "+ merged['content']
print(merged['content'][200])
all_words = []
for row in merged['content']:
  all_words += row.split()
vocab_len = len(set(all_words))
print(all_words[0:10])
print("vocab_len: ",vocab_len)


headline:  মিয়ানমারের ওপর অর্থনৈতিক নিষেধাজ্ঞা আরোপ করতে পারে ইইউ  content  রোহিঙ্গা নির্যাতন ইস্যুতে মিয়ানমারের ওপর অর্থনৈতিক নিষেধাজ্ঞা আরোপের বিষয়ে বিবেচনা করবে ইউরোপীয় ইউনিয়ন। বুধবার এ তথ্য জানিয়েছেন ইইউ'র তিন জ্যেষ্ঠ কর্মকর্তা। তবে এ বিষয়ে এখনো চূড়ান্ত কোন সিদ্ধান্ত নেয়নি ইউরোপের প্রতিনিধিত্বকারী বৃহৎ জোট ইইউ। মিয়ানমারের তৈরী পোষাকসহ বিভিন্ন রপ্তানিমুখী পণ্যের ওপর এ নিষেধাজ্ঞা আরোপ করা হতে পারে বলে ধারণা করা হচ্ছে। এতে করে মিয়ানমারের পণ্য, ইউরোপের বাজারে শুল্কমুক্ত প্রবেশাধিকার হারাবে। এর আগে রোহিঙ্গা মুসলিমদের ওপর নির্বিচারে নির্যাতনের অভিযোগে, মিয়ানমার সেনাবাহিনীর ওপর নিষেধাজ্ঞা আরোপ করে ইউরোপীয় ইউনিয়ন।
মিয়ানমারের ওপর অর্থনৈতিক নিষেধাজ্ঞা আরোপ করতে পারে ইইউ রোহিঙ্গা নির্যাতন ইস্যুতে মিয়ানমারের ওপর অর্থনৈতিক নিষেধাজ্ঞা আরোপের বিষয়ে বিবেচনা করবে ইউরোপীয় ইউনিয়ন। বুধবার এ তথ্য জানিয়েছেন ইইউ'র তিন জ্যেষ্ঠ কর্মকর্তা। তবে এ বিষয়ে এখনো চূড়ান্ত কোন সিদ্ধান্ত নেয়নি ইউরোপের প্রতিনিধিত্বকারী বৃহৎ জোট ইইউ। মিয়ানমারের তৈরী পোষাকসহ বিভিন্ন রপ্তানিমুখী পণ্যের ওপর এ নিষেধাজ্ঞা আরোপ করা হতে পারে 

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(merged['content'], merged['label'], test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
x_train[0]

(46782,)
(11696,)
(46782,)
(11696,)


'‘ঘরে ডেকে এনে জোর করে চুমু খাওয়ার চেষ্টা করেছিল অনুপ’ বলিউড অভিনেতা সালমান খানের উপস্থাপনায় ‘বিগ বস’র দ্বাদশ মৌসুম চলছে। সব সময় আলোচনা-সমালোচনার মধ্যে থাকা এ অনুষ্ঠানকে ঘিরে নতুন করে আলোচনায় এসেছেন বিখ্যাত ভজন গায়ক অনুপ জলোটা। তার বিরুদ্ধে যৌন হেনস্থার অভিযোগ আসছে একের পর এক।\xa0 এবার এক ইসরাইলি মডেল রিনা গোলানের অভিযোগ নিয়ে নতুন করে বিতর্ক তৈরি হল। রিনা গোলান ২০১১ সালে একটি বই লিখেছিলেন। সেই বইতেই ভজন গায়কের উপরে কাস্টিং কাউচের অভিযোগের কথা উল্লেখ করেছিলেন। তিনি লিখেছিলেন, নিজের ঘরে ডেকে এনে তাকে জোর করে চুমু খাওয়ার\xa0চেষ্টা করেছিলেন অনুপ জালোটা। এই অভিযোগ অনুপ জালোটা সম্পূর্ণ ভাবে অস্বীকার করলেও সোশ্যাল মিডিয়াতে এখন এই খবর ভাইরাল। অন্যদিকে জসলিন ও অনুপ জালোটার সম্পর্কের জল গড়িয়েছে অনেক দূর।\xa0 এর আগে, অনীশা সিংহ শর্মা নামের এক মডেল অনুপের বিরুদ্ধে যৌন হেনস্থার অভিযোগ তুলেছেন বলে জানিয়েছে ভারতীয় সংবাদমাধ্যমগুলো। তার অভিযোগ, বর্ষিয়ান এ গায়ক তার সঙ্গে জুটি বেঁধে ‘বিগ বস’-এ প্রতিদ্বন্দ্বিতা করার প্রতিশ্রুতি দিয়ে যৌন মিলনে লিপ্ত হয়েছেন। গত ছ’মাসে তার সঙ্গে অনুপ দু’বার মিলিত হয়েছেন বলে

In [ ]:
preprocessed_train=cleaning(x_train.copy())
preprocessed_train = preprocessed_train.apply(lambda x: "".join(" "+item for item in x))
print(preprocessed_train)

0          ঘর ড এন জোর চুমু খাওয় কর অনুপ  বলিউড অভিনেতা...
1         যুক্তরাজ্য স্বাধীনত দা উত্তাল স্কটল্যান্ড যুক...
2         বগুড়া হিন্দু শিক্ষার্থী গোমাংস খ দেওয় অভিযোগ ...
3         অক্টোবর হঠাৎ উত্তাপ একাদশ জাতী সংসদ নির্বাচন ...
4         সৌম্যর এশিয়া কাপ মাঝ পথ ডাক পা দুবা গ সুযোগ প...
                               ...                        
46777     কর্মদিবস  টাক প্রকল্প স্বাক্ষর রাসিক মেয়র কর্...
46778      ট্রাফিক সপ্তাহ কাঙ্ক্ষিত সফলতা আসেনি  মাসব্য...
46779     খাগড়াছড়ি অনূর্ধ্ব বঙ্গবন্ধু গোল্ডকাপ অনুষ্ঠিত...
46780     জামালপুর ডোবা বৃদ্ধ লাশ উদ্ধ জামালপুর সদর উপজ...
46781     এশিয়া কাপ আফগান বোর্ড প্রধান পদত্যাগ চল এশিয়া...
Name: content, Length: 46782, dtype: object


In [ ]:
preprocessed_test = cleaning(x_test.copy())
preprocessed_test = preprocessed_test.apply(lambda x: "".join(" "+item for item in x))
print(preprocessed_test)

0         সিরাজদিখান মনোনয়ন প্রত্যাশীর জনসভা অনুষ্ঠিত  ...
1         ঢাবির  ক  ইউনিট ভর্তি পরীক্ষা  শতাংশ পাশ ঢাকা...
2         জাকির নায়েক  অফিস এনআইএ  হানা কাগজ অনলাইন ডেস...
3         সেঞ্চুরির নেশা পে কোহ ভারত হ খেলা মান এক ম্যা...
4         আড়াইহাজার শ্বাসরোধ স্ত্রী হত্য অভিযোগ স্টাফ র...
                               ...                        
11691     টস হের ব্যাটিং বাংলাদেশ ম্যাচ শ্রীলঙ্ক বিপক্ষ...
11692      সহানুভূতি দেখা অপমানিত হ আস  জাতী সংসদ ভবন এ...
11693      কো সংস্ক কমি সুপারিশ মুক্তিযুদ্ধ চেতনাবিরোধ...
11694     জাবির   ইউনিট ভর্তি পরীক্ষা জা প্রতিনিধি জাহা...
11695     বিএনপিজামায়াত পুনর্বাসন জন্য ঐক্য ইনু বৃহত্তর...
Name: content, Length: 11696, dtype: object


# Creating Embedding matrix

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

tfidf_vectorizer=TfidfVectorizer()
train_vector=tfidf_vectorizer.fit_transform(preprocessed_train) #tfidf returns csr matrix which keras can't work with
test_vector=tfidf_vectorizer.transform(preprocessed_test)
print("train vector shape: ",train_vector.shape)
print("test vector shape: ", test_vector.shape)


train vector shape:  (46782, 18340)
test vector shape:  (11696, 18340)


In [ ]:
# load the whole embedding into memory
# embeddings_index = dict()
embedding_matrix = []

f = open('/content/drive/MyDrive/FakeNews/bn_glove.39M.100d.txt')
for line in f:
  values = line.split()
  # word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  # embeddings_index[word] = coefs
  embedding_matrix.append(coefs)
f.close()
embedding_matrix = np.array(embedding_matrix)
print('Loaded %s word vectors.' % len(embedding_matrix))
print('shape of embeddings matrix: ',embedding_matrix.shape)

Loaded 178153 word vectors.
shape of embeddings matrix:  (178153, 100)


In [ ]:

# create a weight matrix for words in training docs
# embedding_matrix = zeros((vocab_size, 100))
# for word, i in t.word_index.items():
# 	embedding_vector = embeddings_index.get(word)
# 	if embedding_vector is not None:
# 		embedding_matrix[i] = embedding_vector
# # create a weight matrix for words in training docs
# embedding_matrix = zeros((vocab_size, 100))
# for word, i in t.word_index.items():
# 	embedding_vector = embeddings_index.get(word)
# 	if embedding_vector is not None:
# 		embedding_matrix[i] = embedding_vector

# Running the model

In [ ]:
"""
Going back to home to create a directory for storing checkpoints
"""
!cd .. ;mkdir checkpoints

In [ ]:
print(type(train_vector))
print(type(preprocessed_train))
train_vector=train_vector[:,0:512].toarray()
test_vector=test_vector[:,0:512].toarray()
print(train_vector.shape)
print(test_vector.shape)

<class 'scipy.sparse.csr.csr_matrix'>
<class 'pandas.core.series.Series'>
(46782, 512)
(11696, 512)


In [ ]:
"""
model run
"""
# pickle_load=open('pickle_FullTrain_Guard_Nyt_1_100dim.pickle','rb')
# X,y,embedding_matrix=pickle.load(pickle_load)

# pickle_load = open('pickle_Valid_Mfull_1_100dim.pickle', 'rb')
# X_test, y_test = pickle.load(pickle_load)

vocabulary_size = embedding_matrix.shape[0]
embedding_size=100

kfold = KFold(n_splits=5, shuffle=True, random_state=42)##################
cvscores_Mfull=[]
classfication_report=[]

Fold = 1
for train, val in kfold.split(train_vector, y_train):
    gc.collect()
    K.clear_session()
    print('Fold: ', Fold)

    X_train = train_vector[train]
    X_val = train_vector[val]

    Y_train = y_train[train]
    Y_val = y_train[val]

    print("Initializing Callback :/...")
    #This is where the best model is saved by the callback (cb)
    model_name = '/content/checkpoints/Model_cv_bi_lstm_Fulltrain_1_Callbacks_kfold_'+str(Fold)+'.h5'
    cb = callback(model_name=model_name) 

    # create model
    print("Creating and Fitting Model...")
    model = create_model(vocabulary_size=vocabulary_size,embedding_size=embedding_size,embedding_matrix=embedding_matrix)

    history=model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=10, batch_size=128,shuffle=True,callbacks=cb)##############
    ######################### #callback chilo

    # Save each fold model
    print("Saving Model...")
    model_name = '/content/checkpoints/Model_cv_bi_lstm_Fulltrain_1_kfold_' + str(Fold) + '.h5'########################################3
    model.save(model_name)#################################################
    '''
    model = load_model('Models/Bi_LSTM/Cross_Validation/Model_cv_bi_lstm_Fulltrain_1_kfold_' + str(Fold) + '.h5')
    model.name='Model_lstm_Fulltrain_1.h5'
    '''

    # evaluate the model
    print("Evaluating Model...")
    ##########################################
    scores = model.evaluate(test_vector, y_test, verbose=0)
    print("Eval with Fake or Real %s: %.2f%%" % (model.metrics_names[1], scores[1]))
    cvscores_Mfull.append(scores[1])

    from sklearn.metrics import precision_recall_fscore_support, classification_report

    y_pred = model.predict_classes(test_vector)
    classfication_report.append(classification_report(y_test, y_pred))
    #print('Classification report:\n', classification_report(y_test, y_pred))
    # print('Classification report:\n',precision_recall_fscore_support(y_test,y_pred))
    # print(y_pred)

    '''#######################################################
    ########### Saving Graph ####################
    print("Saving graph...")

    plot_loss_accu(history,'Graphs/Train_Val_Loss_Fold_'+str(Fold)+'.png','Graphs/Train_Val_Acc_Fold_'+str(Fold)+'.png')
    #######################################################'''

    Fold = Fold + 1

print("Accuracy list of Fake or Real: ",cvscores_Mfull)
print("%s: %.2f%%" % ("Mean Accuracy of Fake or Real: ", np.mean(cvscores_Mfull)))
print("%s: %.2f%%" % ("Standard Deviation of Fake or Real: +/-", np.std(cvscores_Mfull)))


print('Classfication Report:')
for cr in classfication_report:
    print(cr)

Fold:  1
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         17815300  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 17,976,301
Trainable params: 161,001
Non-trainable params: 17,815,300
_________________________________________________________________
Epoch 1/10
293/293 [==============================] - 741s 2s/step - loss: 0.1928 - accuracy: 0.9553 - val_loss: 0.1822 - val_accuracy: 0.9554
Epoch 2/10
293/293 [==============================] - 714s 2s/step - loss: 0.1799 - accuracy: 0.9565 - val_loss: 0.1823 - val_accuracy: 0.9554
Epoch 3/10
293/

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold:  2
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         17815300  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 17,976,301
Trainable params: 161,001
Non-trainable params: 17,815,300
_________________________________________________________________
Epoch 1/10
293/293 [==============================] - 723s 2s/step - loss: 0.1959 - accuracy: 0.9374 - val_loss: 0.1792 - val_accuracy: 0.9564
Epoch 2/10
293/293 [==============================] - 739s 3s/step - loss: 0.1881 - accuracy: 0.9536 - val_loss: 0.1792 - val_accuracy: 0.9564
Epoch 3/10
293/